In [150]:
import requests
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import time
import datetime as dt

print('Imports completed!')

Imports completed!


In [151]:
# Read in game score data
game_df = pd.read_excel('./data/apiGameScores.xlsx')
game_df

,game_ids,date,visitor,home,visitor_points,home_points
0,319,2015-11-25T01:00:00.000Z,Boston Celtics,Atlanta Hawks,97.0,121.0
1,1300,2016-04-09T23:30:00.000Z,Boston Celtics,Atlanta Hawks,107.0,118.0
2,1344,2016-04-16T23:00:00.000Z,Boston Celtics,Atlanta Hawks,101.0,102.0
3,1353,2016-04-19T23:00:00.000Z,Boston Celtics,Atlanta Hawks,72.0,89.0
4,1376,2016-04-27T00:30:00.000Z,Boston Celtics,Atlanta Hawks,83.0,110.0
...,...,...,...,...,...,...
14237,9048,2021-04-13T01:00:00.000Z,Washington Wizards,Utah Jazz,125.0,121.0
14238,10012,2021-12-19T02:00:00.000Z,Washington Wizards,Utah Jazz,109.0,103.0
14239,11527,2022-12-23T02:00:00.000Z,Washington Wizards,Utah Jazz,112.0,120.0
14240,13430,2024-03-05T02:00:00.000Z,Washington Wizards,Utah Jazz,115.0,127.0


In [152]:
# Grab game ids to be used with api call
game_ids = game_df['game_ids'].values
game_ids

array([  319,  1300,  1344, ..., 11527, 13430, 14741], dtype=int64)

In [ ]:
url = "https://api-nba-v1.p.rapidapi.com/games/statistics"
headers = {
	"x-rapidapi-key": "dda6cf782dmshebe3119a485b548p154863jsn7c9c31f1e7a7",
	"x-rapidapi-host": "api-nba-v1.p.rapidapi.com"
}

statistics = []
for game_id in game_ids:
	querystring = {"id": game_id}
	response = requests.get(url, headers=headers, params=querystring)
	
 	# Check if the API request was successful
	if response.status_code == 200:
		game_statistics = response.json()
		
		if len(game_statistics.get('response', [])) > 1:
			# Extract game id
			id = game_statistics['parameters']['id']

			# Extract visitor and home team names
			visitor = game_statistics['response'][0].get('team', {}).get('name', 'Unknown')
			home = game_statistics['response'][1].get('team', {}).get('name', 'Unknown')

			# Extract statistics for visitor team
			visitor_statics = game_statistics['response'][0]['statistics'][0]
			visitor_stats = {f"visitor_{key}":value for key, value in visitor_statics.items()}
			visitor_stats['game_id'] = id
			visitor_stats['visitor_team'] = visitor
			
			# Extract statistics for home team
			home_statics = game_statistics['response'][1]['statistics'][0]
			home_stats = {f"home_{key}":value for key, value in home_statics.items()}
			home_stats['game_id'] = id
			home_stats['home_team'] = home

			# Append both visitor and home stats
			statistics.append({**visitor_stats, **home_stats})

# Create the Data Frame
game_statistics_df = pd.DataFrame(statistics)

In [ ]:
# Display
game_statistics_df